In [1]:
!pip install datasets
!pip install tensorflow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 8.4 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.20.3
    Uninstalling huggingface-hub-0.20.3:
      Successfully uninstalled huggingface-hub-0.20.3


In [2]:
import tensorflow as tf
import transformers

from datasets import load_dataset
from transformers import WhisperProcessor, WhisperFeatureExtractor, TFWhisperForConditionalGeneration, WhisperTokenizer

In [3]:
feature_extractor = WhisperFeatureExtractor.from_pretrained("openai/whisper-tiny.en")
tokenizer = WhisperTokenizer.from_pretrained("openai/whisper-tiny.en", predict_timestamps=True)
processor = WhisperProcessor(feature_extractor, tokenizer)
model = TFWhisperForConditionalGeneration.from_pretrained("openai/whisper-tiny.en")
# Loading dataset
ds = load_dataset("hf-internal-testing/librispeech_asr_dummy", "clean", split="validation")

inputs = feature_extractor(
    ds[0]["audio"]["array"], sampling_rate=ds[0]["audio"]["sampling_rate"], return_tensors="tf"
)
input_features = inputs.input_features

# Generating Transcription
generated_ids = model.generate(input_features=input_features)
print(generated_ids)
transcription = processor.tokenizer.decode(generated_ids[0])
print(transcription)

# Save the model
model.save('/content/tf_whisper_saved')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/185k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/805 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.41M [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/1.94k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/151M [00:00<?, ?B/s]

All PyTorch model weights were used when initializing TFWhisperForConditionalGeneration.

All the weights of TFWhisperForConditionalGeneration were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFWhisperForConditionalGeneration for predictions without further training.
/usr/local/lib/python3.10/dist-packages/datasets/load.py:1486: FutureWarning: The repository for hf-internal-testing/librispeech_asr_dummy contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/hf-internal-testing/librispeech_asr_dummy
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


Generating validation split: 0 examples [00:00, ? examples/s]

tf.Tensor(
[[50257 50362  1770    13  2264   346   353   318   262 46329   286   262
   3504  6097    11   290   356   389  9675   284  7062   465 21443    13
  50256]], shape=(1, 25), dtype=int32)
<|startoftranscript|><|notimestamps|> Mr. Quilter is the apostle of the middle classes, and we are glad to welcome his gospel.<|endoftext|>


In [5]:
# model.save('tf_whisper_saved.h5')

In [5]:
class GenerateModel(tf.Module):
  def __init__(self, model):
    super(GenerateModel, self).__init__()
    self.model = model

  @tf.function(
    input_signature=[
      tf.TensorSpec((1, 80, 3000), tf.float32, name="input_features"),
    ],
  )
  def serving(self, input_features):
    outputs = self.model.generate(
      input_features,
      # change below if you think your output will be bigger
      # aka if you have bigger transcriptions
      # you can make it 200 for example
      max_new_tokens=100,
      return_dict_in_generate=True,
    )
    return {"sequences": outputs["sequences"]}

saved_model_dir = '/content/tf_whisper_saved'
tflite_model_path = 'whisper_english.tflite'

generate_model = GenerateModel(model=model)
tf.saved_model.save(generate_model, saved_model_dir, signatures={"serving_default": generate_model.serving})

# Convert the model
converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_dir)
converter.target_spec.supported_ops = [
  tf.lite.OpsSet.TFLITE_BUILTINS, # enable TensorFlow Lite ops.
  tf.lite.OpsSet.SELECT_TF_OPS # enable TensorFlow ops.
]
# Learn about post training quantization
# https://www.tensorflow.org/lite/performance/post_training_quantization

# Dynamic range quantization which reduces the size of the model to 25%
converter.optimizations = [tf.lite.Optimize.DEFAULT]
# Float16 quantization reduces the size to 50%
#converter.target_spec.supported_types = [tf.float16]
tflite_model = converter.convert()

# Save the model
with open(tflite_model_path, 'wb') as f:
    f.write(tflite_model)

In [6]:
import os
quantized_model_size = os.path.getsize('whisper_english.tflite')

print(f"Quantized model size: {quantized_model_size / 1024:.2f} KB")
print(f"Quantized model size: {quantized_model_size / 1024 / 1024:.2f} MB")


Quantized model size: 39425.78 KB
Quantized model size: 38.50 MB


In [7]:
# loaded model... now with generate!
tflite_model_path = 'whisper_english.tflite'
interpreter = tf.lite.Interpreter(tflite_model_path)

tflite_generate = interpreter.get_signature_runner()
generated_ids = tflite_generate(input_features=input_features)["sequences"]
print(generated_ids)
transcription = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]
print(transcription)

[[50257 50362   262   262   262   262   262   262   262   262   262   262
    262   262   262   262   262   262   262   262   262   262   262   262
    262   262   262   262   262   262   262   262   262   262   262   262
    262   262   262   262   262   262   262   262   262   262   262   262
    262   262   262   262   262   262   262   262   262   262   262   262
    262   262   262   262   262   262   262   262   262   262   262   262
    262   262   262   262   262   262   262   262   262   262   262   262
    262   262   262   262   262   262   262   262   262   262   262   262
    262   262   262   262   262]]
 the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the t

In [12]:
# Test another input
from IPython.display import Audio

input_index = 1

display(Audio(data=ds[input_index]["audio"]["array"], rate=ds[input_index]["audio"]["sampling_rate"], autoplay=True))

inputs = feature_extractor(
    ds[input_index ]["audio"]["array"], sampling_rate=ds[input_index ]["audio"]["sampling_rate"], return_tensors="tf"
)
input_features = inputs.input_features

print("Base Model ... ")
generated_ids = model.generate(input_features=input_features)
print(generated_ids)
transcription = processor.tokenizer.decode(generated_ids[0])
print(transcription)

print("Tflite Model ... ")


generated_ids = tflite_generate(input_features=input_features)["sequences"]
print(generated_ids)
transcription = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]
print(transcription)

Base Model ... 
tf.Tensor(
[[50257 50362  5414   318  1770    13  2264   346   353   338  5642  1342
   3499   621   465  2300    13 50256]], shape=(1, 18), dtype=int32)
<|startoftranscript|><|notimestamps|> Nor is Mr. Quilter's manner less interesting than his matter.<|endoftext|>
Tflite Model ... 
[[50257 50362   262   262   262   262   262   262   262   262   262   262
    262   262   262   262   262   262   262   262   262   262   262   262
    262   262   262   262   262   262   262   262   262   262   262   262
    262   262   262   262   262   262   262   262   262   262   262   262
    262   262   262   262   262   262   262   262   262   262   262   262
    262   262   262   262   262   262   262   262   262   262   262   262
    262   262   262   262   262   262   262   262   262   262   262   262
    262   262   262   262   262   262   262   262   262   262   262   262
    262   262   262   262   262]]
 the the the the the the the the the the the the the the the the the the 

In [31]:
!pip install ffmpeg-python
!pip install librosa

In [35]:
import librosa


def test_input(audio, sr):
    input_audio = audio
    if (sr != 16000):
        print(f"Resampling Audio From {sr} to 16000")
        input_audio = librosa.resample(audio.astype(float), orig_sr=sr, target_sr=16000)

    display(Audio(data=input_audio, rate=16000, autoplay=True))
    inputs = feature_extractor(
        input_audio, sampling_rate=16000, return_tensors="tf"
    )
    input_features = inputs.input_features
    print("Base Model ... ")
    generated_ids = model.generate(input_features=input_features)
    print(generated_ids)
    transcription = processor.tokenizer.decode(generated_ids[0])
    print(transcription)

    print("Tflite Model ... ")


    generated_ids = tflite_generate(input_features=input_features)["sequences"]
    print(generated_ids)
    transcription = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]
    print(transcription)


In [27]:
# Code to record our own input
from IPython.display import HTML
from google.colab.output import eval_js
from base64 import b64decode
import ffmpeg
from scipy.io.wavfile import read as wav_read
import io

AUDIO_HTML = """
<script>
var my_div = document.createElement("DIV");
var my_p = document.createElement("P");
var my_btn = document.createElement("BUTTON");
var t = document.createTextNode("Press to start recording");

my_btn.appendChild(t);
//my_p.appendChild(my_btn);
my_div.appendChild(my_btn);
document.body.appendChild(my_div);

var base64data = 0;
var reader;
var recorder, gumStream;
var recordButton = my_btn;

var handleSuccess = function(stream) {
  gumStream = stream;
  var options = {
    //bitsPerSecond: 8000, //chrome seems to ignore, always 48k
    mimeType : 'audio/webm;codecs=opus'
    //mimeType : 'audio/webm;codecs=pcm'
  };
  //recorder = new MediaRecorder(stream, options);
  recorder = new MediaRecorder(stream);
  recorder.ondataavailable = function(e) {
    var url = URL.createObjectURL(e.data);
    var preview = document.createElement('audio');
    preview.controls = true;
    preview.src = url;
    document.body.appendChild(preview);

    reader = new FileReader();
    reader.readAsDataURL(e.data);
    reader.onloadend = function() {
      base64data = reader.result;
      //console.log("Inside FileReader:" + base64data);
    }
  };
  recorder.start();
  };

recordButton.innerText = "Recording... press to stop";

navigator.mediaDevices.getUserMedia({audio: true}).then(handleSuccess);


function toggleRecording() {
  if (recorder && recorder.state == "recording") {
      recorder.stop();
      gumStream.getAudioTracks()[0].stop();
      recordButton.innerText = "Saving the recording... pls wait!"
  }
}

// https://stackoverflow.com/a/951057
function sleep(ms) {
  return new Promise(resolve => setTimeout(resolve, ms));
}

var data = new Promise(resolve=>{
//recordButton.addEventListener("click", toggleRecording);
recordButton.onclick = ()=>{
toggleRecording()

sleep(2000).then(() => {
  // wait 2000ms for the data to be available...
  // ideally this should use something like await...
  //console.log("Inside data:" + base64data)
  resolve(base64data.toString())

});

}
});

</script>
"""

def get_audio():
  display(HTML(AUDIO_HTML))
  data = eval_js("data")
  binary = b64decode(data.split(',')[1])

  process = (ffmpeg
    .input('pipe:0')
    .output('pipe:1', format='wav')
    .run_async(pipe_stdin=True, pipe_stdout=True, pipe_stderr=True, quiet=True, overwrite_output=True)
  )
  output, err = process.communicate(input=binary)

  riff_chunk_size = len(output) - 8
  # Break up the chunk size into four bytes, held in b.
  q = riff_chunk_size
  b = []
  for i in range(4):
      q, r = divmod(q, 256)
      b.append(r)

  # Replace bytes 4:8 in proc.stdout with the actual size of the RIFF chunk.
  riff = output[:4] + bytes(b) + output[8:]

  sr, audio = wav_read(io.BytesIO(riff))      # returns sample rate, data from wav file

  return audio, sr

print("Chrome Audio Recorder Defined")


Chrome Audio Recorder Defined


In [28]:
audio_1, sr_1 = get_audio()
print("DONE")

DONE


In [36]:
test_input(audio_1, sr_1)

Resampling Audio From 48000 to 16000


Base Model ... 
tf.Tensor(
[[50257 50362   314  1101  1016   284   467   284   262  1306   530    13
  50256]], shape=(1, 13), dtype=int32)
<|startoftranscript|><|notimestamps|> I'm going to go to the next one.<|endoftext|>
Tflite Model ... 
[[50257 50362   484   821   821   821   821   821   821   821   821   821
    821   821   821   821   821   821   821   821   821   821   821   821
    821   484   484   484   484   484   484   484   484   484   484   484
    484   484   484   484   484   484   484   484   484   484   484   484
    484   484   484   484   484   484   821   821   821   821   821   821
    821   821   821   821   821   821   821   821   821   821   821   821
    821   821   821   821   821   821   821   821   821   821   821   821
    821   821   821   821   821   821   821   821   821   821   821   821
    821   821   821   821   821]]
 they're're're're're're're're're're're're're're're're're're're're're're they they they they they they they they they they they they 